In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import seaborn as sns

# 1. CẤU HÌNH
sns.set_theme(style="whitegrid")
pd.set_option('display.float_format', lambda x: '%.2f' % x)

BASE_DIR = '..'
CITY_FOLDER = 'paris'  
PROCESSED_DIR = os.path.join(BASE_DIR, 'processed', CITY_FOLDER)

print(f"--- BẮT ĐẦU TÍNH KPI CHO: {CITY_FOLDER.upper()} ---")

# 2. HÀM TÍNH TOÁN KPI (ĐÃ TỐI ƯU BỘ NHỚ)
def calculate_snapshot_kpi(snapshot_path):
    snapshot_name = os.path.basename(snapshot_path)
    print(f"\n>> Đang xử lý snapshot: {snapshot_name}")
    
    # --- BƯỚC 1: Xử lý Listings trước ---
    try:
        listings_path = os.path.join(snapshot_path, 'listings_processed.csv')
        df_list = pd.read_csv(listings_path, low_memory=False)
    except FileNotFoundError:
        print("   [!] Không tìm thấy file listings. Bỏ qua.")
        return None, None, None

    # Lọc dữ liệu sạch (QA Filter)
    valid_listings = df_list[
        (df_list['qa_flag_price_zero'] == False) & 
        (df_list['qa_flag_out_of_city'] == False)
    ].copy()
    
    # Lấy danh sách ID hợp lệ để lọc Calendar
    valid_ids = set(valid_listings['id'].unique()) # Dùng set để tra cứu nhanh hơn

    # --- BƯỚC 2: Xử lý Calendar theo CHUNK (Từng phần nhỏ) ---
    cal_path = os.path.join(snapshot_path, 'calendar_processed.csv')
    if not os.path.exists(cal_path):
        print("   [!] Không tìm thấy file calendar. Bỏ qua.")
        return None, None, None

    # Đọc Calendar từng 1 triệu dòng một -> Lọc ngay -> Gom lại
    chunk_list = []
    # Chỉ đọc các cột cần thiết để tiết kiệm RAM tối đa
    use_cols = ['listing_id', 'date', 'available'] 
    
    try:
        # chunksize=1000000: Đọc mỗi lần 1 triệu dòng
        with pd.read_csv(cal_path, usecols=use_cols, chunksize=1000000, low_memory=False) as reader:
            for chunk in reader:
                # Chỉ giữ lại dòng thuộc về các listing hợp lệ
                filtered_chunk = chunk[chunk['listing_id'].isin(valid_ids)]
                chunk_list.append(filtered_chunk)
        
        # Gộp các mảnh nhỏ lại thành 1 file hoàn chỉnh (nhưng nhỏ hơn nhiều)
        valid_cal = pd.concat(chunk_list, ignore_index=True)
        del chunk_list # Giải phóng bộ nhớ đệm
        
    except Exception as e:
        print(f"   [!] Lỗi khi đọc calendar: {e}")
        return None, None, None

    # ==============================================================================
    # NHÓM 1: CUNG & HOẠT ĐỘNG
    # ==============================================================================
    total_supply = len(valid_listings)
    
    host_counts = valid_listings['host_id'].value_counts()
    multi_host_rate = (host_counts[host_counts > 1].sum() / total_supply) * 100 if total_supply > 0 else 0

    room_stats = valid_listings['room_type'].value_counts(normalize=True).reset_index()
    room_stats.columns = ['room_type', 'percentage']
    room_stats['percentage'] = room_stats['percentage'] * 100
    room_stats['snapshot_date'] = snapshot_name

    # ==============================================================================
    # NHÓM 2: GIÁ THUÊ (PRICE)
    # ==============================================================================
    median_price = valid_listings['price_numeric'].median()
    
    price_by_room = valid_listings.groupby('room_type')['price_numeric'].median().reset_index()
    price_by_room.rename(columns={'price_numeric': 'median_price'}, inplace=True)
    price_by_room['snapshot_date'] = snapshot_name

    # ==============================================================================
    # NHÓM 3: TÍNH SẴN SÀNG (AVAILABILITY & OCCUPANCY)
    # ==============================================================================
    median_avail_90 = valid_listings['availability_90'].median()
    
    # Tính Occupancy Rate từ valid_cal (đã được thu nhỏ)
    valid_cal['date'] = pd.to_datetime(valid_cal['date'])
    valid_cal['is_booked'] = valid_cal['available'].apply(lambda x: 1 if x == 'f' else 0)
    
    daily_occupancy = valid_cal.groupby('date')['is_booked'].mean().reset_index()
    daily_occupancy.columns = ['date', 'occupancy_rate']
    daily_occupancy['snapshot_date'] = snapshot_name

    # --- Đóng gói kết quả Summary ---
    summary_row = {
        'snapshot_date': snapshot_name,
        'total_listings': total_supply,
        'multi_host_rate': multi_host_rate,
        'median_price': median_price,
        'median_avail_90': median_avail_90
    }
    
    return summary_row, room_stats, daily_occupancy

# 3. CHẠY VÒNG LẶP & TỔNG HỢP
all_snapshots = glob.glob(os.path.join(PROCESSED_DIR, '*'))
summary_list = []
room_stats_list = []
occupancy_list = []

for folder in all_snapshots:
    if os.path.isdir(folder):
        res_summary, res_room, res_occ = calculate_snapshot_kpi(folder)
        if res_summary:
            summary_list.append(res_summary)
            room_stats_list.append(res_room)
            occupancy_list.append(res_occ)

# 4. GỘP & LƯU FILE
if summary_list:
    # 4.1 Bảng Tổng hợp Chính
    df_summary = pd.DataFrame(summary_list).sort_values('snapshot_date')
    df_summary.to_csv(os.path.join(PROCESSED_DIR, f'kpi_summary_general_{CITY_FOLDER}.csv'), index=False)
    
    # 4.2 Bảng Chi tiết Room Type
    room_stats_list = [df for df in room_stats_list if not df.empty]
    if room_stats_list:
        df_room_all = pd.concat(room_stats_list, ignore_index=True)
        df_room_all.to_csv(os.path.join(PROCESSED_DIR, f'kpi_room_type_{CITY_FOLDER}.csv'), index=False)
    
    # 4.3 Bảng Seasonality
    occupancy_list = [df for df in occupancy_list if not df.empty]
    if occupancy_list:
        df_occ_all = pd.concat(occupancy_list, ignore_index=True)
        df_occ_all.to_csv(os.path.join(PROCESSED_DIR, f'kpi_seasonality_{CITY_FOLDER}.csv'), index=False)

    print("\n--- KẾT QUẢ ---")
    print("1. Bảng tổng hợp chung (5 dòng đầu):")
    print(df_summary.head())
    print(f"\n✅ Đã lưu 3 file KPI vào: {PROCESSED_DIR}")
else:
    print("Không có dữ liệu để tính KPI.")

--- BẮT ĐẦU TÍNH KPI CHO: PARIS (CHẾ ĐỘ TIẾT KIỆM RAM) ---

>> Đang xử lý snapshot: 03 March, 2025

>> Đang xử lý snapshot: 06 December, 2024

>> Đang xử lý snapshot: 06 June, 2025

>> Đang xử lý snapshot: 12 September, 2025

--- KẾT QUẢ ---
1. Bảng tổng hợp chung (5 dòng đầu):
        snapshot_date  total_listings  multi_host_rate  median_price  \
0      03 March, 2025           55601            39.48        146.00   
1   06 December, 2024           60046            38.23        150.00   
2       06 June, 2025           53912            40.29        161.00   
3  12 September, 2025               0             0.00           NaN   

   median_avail_90  
0            32.00  
1            59.00  
2            44.00  
3              NaN  

✅ Đã lưu 3 file KPI vào: ..\processed\paris
